In [54]:
from tai_chi_tuna.config import PhaseConfig
from tai_chi_tuna.front.html import DOM, list_group_kv
from pathlib import Path

In [18]:
!mkdir -p example1

In [25]:
%%writefile example1/phase.json
{
  "enrich": [
    {
      "src": "image_path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    }
  ],
  "quantify": [
    {
      "src": "image",
      "x": true,
      "kwargs": {
        "mean_": "imagenet",
        "std_": "imagenet"
      },
      "quantify": "QuantifyImage"
    },
    {
      "src": "Genre",
      "x": false,
      "kwargs": {
        "min_frequency": 1,
        "separator": ";"
      },
      "quantify": "QuantifyMultiCategory"
    }
  ],
  "batch_level": {
    "valid_ratio": 0.1,
    "batch_size": 32,
    "shuffle": true,
    "num_workers": 0
  },
  "x_models": {
    "image": {
      "model_name": "ImageConvEncoder",
      "src": "image",
      "kwargs": {
        "name": "resnet34"
      }
    }
  },
  "y_models": {
    "Genre": {
      "model_name": "MultiCategoryTop",
      "src": "Genre",
      "kwargs": {
        "hidden_size": 0
      }
    }
  },
  "task_slug": "image_to_Genre"
}

Overwriting example1/phase.json


In [26]:
phase = PhaseConfig.load(Path("./example1"))

In [59]:
class Narrator:
    def __init__(self, phase):
        self.phase=phase
        
    def __call__(self,):
        self.narrate()()
        
    def title(self, text) -> DOM:
        return DOM(text, "h3",
                   {"style":"color:#FFAA33;background-color:#FFFFFF;padding:3px"})
    
    def empty_step(self,):
        return DOM("","ul",dict())
    
    def new_line(self, inner):
        return DOM(inner,"li",dict(style="padding:2px;background-color:#FFFFDD"))
        
    def narrate(self,):
        config = self.phase.config
        doc = DOM("","div")
        if "enrich" in config:
            step =  self.empty_step()
            step.append(self.title("Step 1: Enrich"))
            enrichs = config['enrich']
            
            for enrich in enrichs:
                src = enrich.get('src')
                enrich_type = enrich.get('enrich')
                step.append(self.new_line(f"select column '{src}' for 'src'"))
                step.append(
                    self.new_line(
                        f"select '{enrich_type}' for 'enrich'"))
                step.append(self.new_line(list_group_kv(enrich.get("kwargs"))))
            step.append(self.new_line("Click ✔️Next above"))
            doc.append(step)
        else:
            step =  self.empty_step()
            step.append(self.title("Step 1: Enrich"))
            step.append(self.new_line("No enrich set, click ✔️Next above"))
            doc.append(step)
        return doc

In [60]:
Narrator(phase)()

convert:,RGB
size:,224
